In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras 
import os 
import math 
import random 
import pickle 
import matplotlib.pyplot as plt 
from collections import deque 

from vehicle_model_DDQN3_1 import Environment 
from cell_model import CellModel 

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
drving_cycle = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
battery_path = "../../OC_SIM_DB/OC_SIM_DB_Bat/OC_SIM_DB_Bat_e-4wd_Battery.mat"
motor_path = "../../OC_SIM_DB/OC_SIM_DB_Mot/OC_SIM_DB_Mot_id_75_110_Westinghouse.mat"
cell_model = CellModel()
env = Environment(cell_model, drving_cycle, battery_path, motor_path, 1)


In [3]:
# STATE_SIZE = env.calculation_comp["state_size"]
STATE_SIZE = 4
ACTION_SIZE = env.calculation_comp["action_size"] 
LEARNING_RATE = 0.00025 

TOTAL_EPISODES = 200
MAX_STEPS = 50000 

GAMMA = 0.95 

MAX_EPSILON = 1 
MIN_EPSILON = 0.01 
DECAY_RATE = 0.00002
BATCH_SIZE = 32 
TAU = 0.001 
DELAY_TRAINING = 3000 
EPSILON_MIN_ITER = 5000

In [4]:
primary_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])
target_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])

primary_network.compile(
    loss="mse", 
    optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
)

# for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
#     t.assign(p)

In [5]:
def update_network(primary_network, target_network): 
    for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
        t.assign(t * (1 - TAU) + p * TAU)

In [6]:
class Memory: 
    def __init__(self, max_memory): 
        self.max_memory = max_memory 
        self._samples = [] 
        
    def add_sample(self, sample): 
        self._samples.append(sample)
        if len(self._samples) > self.max_memory: 
            self._samples.pop(0)
        
    def sample(self, no_samples): 
        if no_samples > len(self._samples): 
            return random.sample(self._samples, len(self._samples))
        else: 
            return random.sample(self._samples, no_samples)
    
    @property
    def num_samples(self):
        return len(self._samples)
    

# memory = Memory(10000)

In [7]:
def choose_action(state, primary_network, eps): 
    if random.random() < eps: 
        return random.randint(0, ACTION_SIZE - 1)
    else: 
        return np.argmax(primary_network(np.array(state).reshape(1, -1))) 

In [8]:
def train(primary_network, target_network, memory): 
    batch = memory.sample(BATCH_SIZE)
    states = np.array([val[0] for val in batch]) 
    actions = np.array([val[1] for val in batch])
    rewards = np.array([val[2] for val in batch])
    next_states = np.array([np.zeros(STATE_SIZE) if val[3] is None else val[3]  
                            for val in batch])
    
    prim_qt = primary_network(states)
    prim_qtp1 = primary_network(next_states)
    target_q = prim_qt.numpy() 
    updates = rewards 
    valid_idxs = next_states.sum(axis=1) != 0 
    batch_idxs = np.arange(BATCH_SIZE)
    prim_action_tp1 = np.argmax(prim_qtp1.numpy(), axis=1)
    q_from_target = target_network(next_states)
    updates[valid_idxs] += GAMMA * q_from_target.numpy()[batch_idxs[valid_idxs], 
                                                        prim_action_tp1[valid_idxs]]
    
    target_q[batch_idxs, actions] = updates 
    loss = primary_network.train_on_batch(states, target_q)
    return loss 
    
    
    

In [9]:
def initialization_with_rewardFactor(reward_factor):
    env = Environment(cell_model, drving_cycle, battery_path, motor_path, reward_factor)
    
    memory = Memory(10000)
    
    primary_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(),  
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    target_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    primary_network.compile(
        loss="mse", 
        optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
    )
    return env, memory, primary_network, target_network 
    

In [10]:
print("environment version: {}".format(env.version)) 

 
reward_factors = [10]
results_dict = {} 

for reward_factor in reward_factors: 
    eps = MAX_EPSILON 
    steps = 0
    episode_rewards = [] 
    episode_SOCs = [] 
    episode_FCs = [] 
    
    env, memory, primary_network, target_network = initialization_with_rewardFactor(reward_factor)
    for episode in range(TOTAL_EPISODES): 
        state = env.reset() 
        avg_loss = 0 
        total_reward = 0
        cnt = 1 

        while True:
            action = choose_action(state, primary_network, eps)
            next_state, reward, done = env.step(action)
            total_reward += reward 
            if done: 
                next_state = None 
            memory.add_sample((state, action, reward, next_state))

            if steps > DELAY_TRAINING: 
                loss = train(primary_network, target_network, memory)
                update_network(primary_network, target_network)
                eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * steps)
            else: 
                loss = -1

            avg_loss += loss 
            steps += 1 

            if done: 
                if steps > DELAY_TRAINING: 
                    SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
                    avg_loss /= cnt 
                    print('Episode: {}'.format(episode + 1),
                          'Total reward: {}'.format(total_reward), 
                          'Explore P: {:.4f}'.format(eps), 
                          "SOC: {:.4f}".format(env.SOC), 
                         "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
                         "Fuel Consumption: {:.4f}".format(env.fuel_consumption), 
                         )
                else: 
                    print(f"Pre-training...Episode: {episode}")
                
                episode_rewards.append(total_reward)
                episode_SOCs.append(env.SOC)
                episode_FCs.append(env.fuel_consumption)
                break 

            state = next_state 
            cnt += 1 
    
    results_dict[reward_factor] = {
        "rewards": episode_rewards, 
        "SOCs": episode_SOCs, 
        "FCs": episode_FCs 
    }
            
    

environment version: 1
maximum steps, simulation is done ... 
Pre-training...Episode: 0
maximum steps, simulation is done ... 
Pre-training...Episode: 1

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer

maximum steps, simulation is done ... 
Episode: 41 Total reward: -609.2107624656998 Explore P: 0.3311 SOC: 0.6946 Cumulative_SOC_deviation: 55.6533 Fuel Consumption: 52.6779
maximum steps, simulation is done ... 
Episode: 42 Total reward: -661.1465562356891 Explore P: 0.3224 SOC: 0.6888 Cumulative_SOC_deviation: 60.9101 Fuel Consumption: 52.0460
maximum steps, simulation is done ... 
Episode: 43 Total reward: -737.3709678733514 Explore P: 0.3140 SOC: 0.6918 Cumulative_SOC_deviation: 68.5207 Fuel Consumption: 52.1640
maximum steps, simulation is done ... 
Episode: 44 Total reward: -889.8529685551639 Explore P: 0.3057 SOC: 0.7371 Cumulative_SOC_deviation: 83.4281 Fuel Consumption: 55.5721
maximum steps, simulation is done ... 
Episode: 45 Total reward: -643.6001427369201 Explore P: 0.2977 SOC: 0.7021 Cumulative_SOC_deviation: 59.0884 Fuel Consumption: 52.7164
maximum steps, simulation is done ... 
Episode: 46 Total reward: -529.6314392468519 Explore P: 0.2899 SOC: 0.6383 Cumulative_SOC_d

maximum steps, simulation is done ... 
Episode: 89 Total reward: -476.70100012026376 Explore P: 0.0960 SOC: 0.6310 Cumulative_SOC_deviation: 42.9809 Fuel Consumption: 46.8920
maximum steps, simulation is done ... 
Episode: 90 Total reward: -461.3508006049922 Explore P: 0.0936 SOC: 0.6463 Cumulative_SOC_deviation: 41.3288 Fuel Consumption: 48.0632
maximum steps, simulation is done ... 
Episode: 91 Total reward: -552.0258518235112 Explore P: 0.0914 SOC: 0.6645 Cumulative_SOC_deviation: 50.1885 Fuel Consumption: 50.1410
maximum steps, simulation is done ... 
Episode: 92 Total reward: -383.5221907439334 Explore P: 0.0892 SOC: 0.6365 Cumulative_SOC_deviation: 33.6183 Fuel Consumption: 47.3395
maximum steps, simulation is done ... 
Episode: 93 Total reward: -501.3416404466421 Explore P: 0.0870 SOC: 0.6929 Cumulative_SOC_deviation: 44.8994 Fuel Consumption: 52.3475
maximum steps, simulation is done ... 
Episode: 94 Total reward: -336.2274290770162 Explore P: 0.0849 SOC: 0.6138 Cumulative_SOC_

maximum steps, simulation is done ... 
Episode: 136 Total reward: -344.71281873834295 Explore P: 0.0336 SOC: 0.6188 Cumulative_SOC_deviation: 29.8878 Fuel Consumption: 45.8348
maximum steps, simulation is done ... 
Episode: 137 Total reward: -465.612725221063 Explore P: 0.0330 SOC: 0.6463 Cumulative_SOC_deviation: 41.7574 Fuel Consumption: 48.0388
maximum steps, simulation is done ... 
Episode: 138 Total reward: -338.07479630914474 Explore P: 0.0324 SOC: 0.6153 Cumulative_SOC_deviation: 29.2689 Fuel Consumption: 45.3853
maximum steps, simulation is done ... 
Episode: 139 Total reward: -339.58643021813003 Explore P: 0.0318 SOC: 0.6187 Cumulative_SOC_deviation: 29.4093 Fuel Consumption: 45.4933
maximum steps, simulation is done ... 
Episode: 140 Total reward: -379.2179069553769 Explore P: 0.0312 SOC: 0.6385 Cumulative_SOC_deviation: 33.2275 Fuel Consumption: 46.9432
maximum steps, simulation is done ... 
Episode: 141 Total reward: -296.64188326456457 Explore P: 0.0306 SOC: 0.6134 Cumulat

maximum steps, simulation is done ... 
Episode: 183 Total reward: -348.08406532820436 Explore P: 0.0165 SOC: 0.6096 Cumulative_SOC_deviation: 30.3187 Fuel Consumption: 44.8973
maximum steps, simulation is done ... 
Episode: 184 Total reward: -345.0161335627402 Explore P: 0.0163 SOC: 0.6164 Cumulative_SOC_deviation: 29.9283 Fuel Consumption: 45.7330
maximum steps, simulation is done ... 
Episode: 185 Total reward: -321.7641369273057 Explore P: 0.0162 SOC: 0.6081 Cumulative_SOC_deviation: 27.6637 Fuel Consumption: 45.1274
maximum steps, simulation is done ... 
Episode: 186 Total reward: -375.1319193094701 Explore P: 0.0160 SOC: 0.6120 Cumulative_SOC_deviation: 32.9625 Fuel Consumption: 45.5064
maximum steps, simulation is done ... 
Episode: 187 Total reward: -385.72317766733516 Explore P: 0.0158 SOC: 0.6096 Cumulative_SOC_deviation: 34.0495 Fuel Consumption: 45.2282
maximum steps, simulation is done ... 
Episode: 188 Total reward: -328.05491696772026 Explore P: 0.0157 SOC: 0.6068 Cumulat

In [13]:
with open("DDQN3_1.pkl", "wb") as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)

In [12]:
# with open("results/replay_memory_size_effect.pkl", "rb") as f: 
#     data = pickle.load(f)
    
# data